[![imagenes/pythonista.png](imagenes/pythonista.png)](https://pythonista.io)

# Servidor API REST.

**ADVERTENCIA:**  

Para poder realizar exitosamente los ejercicios de esta notebook, es necesario haber seguido al pie de la letra y en orden sucesivo las instrucciones de todas las notebooks previas.

## El servicio web localizado en ```api/```.

En el capítulo [```13_operaciones_basicas _con_modelos.ipynb```](13_operaciones_basicas _con_modelos.ipynb) se creó la aplicación ```api``` la cual definió los siguiente *endpoints*:

* ```api/```, el cual regresa un documento en formato *JSON* que contiene un listado de todos los datos almanceados en la tabla ```api_alumno``` la cual está ligada a los objetos instanciados de la clase ```api.models.Alumno```.
* ```api/carga```, la cual poblaría la base de datos ```api_alumno``` a partir de un archivo llamado ```alumnos.json```, localizado en el directorio ```tutorial```.

En este capítulo se definirán *endpoints* que corresponden a la *URL*:

```
api/<clave>
```

* Donde ```<clave>``` es corresponde a un número de 4 dígitos.

Dependiendo del método mediante el cual se accede a dicha *URL*, se pueden realizar las siguientes operaciones.

* Al acceder a la *URL* mediante el método ```GET```, el sevidor regresará un documento *JSON* con el estado del objeto instanciado de ```api.models.Alumno``` cuyo atributo ```numero_de_cuenta``` corresponda al número de la *URL*. En caso de no existir, regresará un estado ```404```.
* Al acceder a la *URL* mediante el método ```POST```, el sevidor intentará crear una instancia de la clase ```api.models.Alumno``` cuyo tributo ```numero_de_cuenta```corresponderá al número de la *URL*. El servidor verificará que los datos enviados tengan la estructura y contenido adecuado. En caso de ser así, regresará un documento *JSON* con el estado del objeto creado. En caso de que ya exista un objeto cuyo atributo clave sea igual al de la *URL* o no tenga los datos o no tenga la estructura adecuada, el servidor regresará un error ```400```.
* Al acceder a la *URL* mediante el método ```DELETE```, el sevidor regresará un documento *JSON* con el estado del objeto instanciado de ```api.models.Alumno``` cuyo atributo ```numero_de_cuenta``` corresponda al número de la *URL* y lo eliminará de la base de datos. En caso de no exisitir, regresará un estado ```404```.


## Reglas de los campos.

* Los campos ```nombre```, ```primer_apellido```, ```segundo_apellido``` y ```carrera``` son cadenas de caracteres.
* El campo ```carrera``` debe corresponder a una carrera válida.
* El campo ```semestre``` es un número entero mayor que cero.
* El campo ```promedio``` es un número que va de 0 a 10.
* El campo ```al_corriente``` es un booleano.
* El campo ```segundo_apellido``` es opcional, todos los demás son obligatorios.

In [ ]:
%cd tutorial

## Definición de las *URLs*.

El archivo ```src/15/urls.py``` contiene lo siguiente:

``` python 
from django.urls import path, re_path
from . import views, endpoint_views

urlpatterns = [path('', views.vista),
               path('carga', views.carga),
               re_path(r'^(?P<clave>[0-9]{4}$)', endpoint_views.clave),]
```

En donde se define una relación entre un patrón que corresponde a una expresión regular que identifca a un número de 4 dígitos y la función ```clave()``` del módulo ```endpoint_views```.

* A continuación se sustituirá el archivo ```tutorial/api/urls.py``` con el archivo ```src/15/ursl.py```.

* Para las plataformas basadas en GNU/Linux o MacOS X.

In [ ]:
!cp ../src/15/urls.py api/urls.py

* Para las plataformas basadas en Windows.

In [ ]:
!copy ..\src\15\urls.py api\urls.py

* La sustitución del archivo ```api/ursl.py``` puede ser verificada desde la siguiente celda.

In [ ]:
%pycat api\urls.py

## El archivo ```endpoint_views.```

Este archivo es contiene a la función de vista ```clave()```, la cual es capaz de procesar los métodos ```GET```, ```POST``` y ```DELETE```.

``` python
from . import models
from django.views.decorators.csrf import csrf_exempt
from django.http import JsonResponse, HttpResponseNotFound, HttpResponseBadRequest

""" El objeto campos define la estructura de los campos aceptables para
los datos relacionando el nombree del campo y una tupla que cuyo primer
elemento es el tipo de dato del que se trata y el segundo elemento es 
un booleano que indica si el campo es obligatorio. """ 
campos = {'numero_de_cuenta':(int, True), 
          'nombre':(str, True), 
          'primer_apellido':(str, True),
          'segundo_apellido':(str, False), 
          'carrera':(str, True), 
          'semestre':(int, True),
          'promedio':(float, True), 
          'al_corriente':(bool, True)}

"""El objeto carreras contiene una tupla con las carreras permitidas en el campo carrera.""" 
carreras = ('Sistemas', 'Derecho', 'Actuaría', 'Arquitectura', 'Administración')

"""El objeto estructura_base es un objeto de tipo set que contiene los campos permitidos."""
estructura_base = set(campos)


def reglas(valor, campo):
    """ Función que valida las reglas de negocio """
    if campo == "carrera" and valor not in carreras:
        return False
    elif campo == "semestre" and valor < 1:
        return False
    elif campo == "promedio" and (valor < 0 or valor > 10):
        return False
    elif (campo in ("nombre", "primer_apellido") and (valor == "")):
        return False
    else:
        return True         

    
def valida(dato, campo):
    """ Función que valida un dato con relación a su campo correspondiente."""
    tipo = campos[campo][0]
    try:
        if tipo is not str:
            valor = eval(dato)
        else:
            valor = dato
        if type(valor) is tipo or (tipo is float and type(dato) is int):
            return reglas(valor, campo)
        else:
            return False
    except:
        return False


@csrf_exempt
def clave(request, clave):
    '''Función de vista que define un endpoint correspondiente a una clave de 4 dígitos.
    Opera con los métodos GET, POST, DELETE.'''
    # Cuando la petición es GET va a obtener los datos del alumno con la clave correrspondiente.
    # Esta operación se realiza en caso de que exista un objeto con el número de cuenta.
    if request.method == "GET":
        try:
            alumno = models.Alumno.objects.get(numero_de_cuenta=clave) 
            return JsonResponse({campo:getattr(alumno, campo) for campo in campos})
        # La excepción models.Alumno.DoesNotExist se desencadena cuando la búsqueda no arroje un resultado.
        except models.Alumno.DoesNotExist:
            return HttpResponseNotFound()
        
    # Cuando la petición es DELETE el alumno es eliminado de la base de datos.
    # Esta operación se realiza en caso de que exista un objeto con el número de cuenta.
    if request.method == "DELETE":
        try:
            alumno = models.Alumno.objects.get(numero_de_cuenta=clave)
            alumno.delete()
            return JsonResponse({'estado': 'eliminado'})   
        except models.Alumno.DoesNotExist:
            return HttpResponseNotFound()
    # Cuando la petición es POST va a dar de alta los datos del alumno con la clave correspondiente y los datos enviados.
    # Esta operación se realiza en caso de que no exista un objeto con el número de cuenta.
    if request.method == "POST":
        try:
            alumno = models.Alumno.objects.get(numero_de_cuenta=clave) 
            return HttpResponseBadRequest()
        except models.Alumno.DoesNotExist:
            registro = request.POST.dict()
            registro["numero_de_cuenta"] = clave
            objeto = models.Alumno()
            estructura_registro = set(registro)
            if estructura_registro.issubset(estructura_base):
                for campo in estructura_base:
                    if valida(registro[campo], campo):
                        if campos[campo][0] is not str:
                            valor = eval(registro[campo])
                        else:
                            valor = registro[campo]
                        setattr(objeto, campo, valor)
                    else:
                        return HttpResponseBadRequest()
                objeto.save()
                return JsonResponse(registro)
            else:
                return HttpResponseBadRequest()
```


* Se copiará el archivo ```src/15/endpoint_views.py``` al directorio ```api\```.

* Para plataformas basadas en GNU/Linux o MacOS X.

In [ ]:
!cp ../src/15/endpoint_views.py api/endpoint_views.py

* Para plataformas basadas en Windows.

In [ ]:
!copy ..\src\15\endpoint_views.py api\endpoint_views.py

* La siguiente celda permite verificar la copia del archivo ```src/15/endpoint_views.py``` al directorio ```api\```.

In [ ]:
%pycat api/endpoint_views.py

**ADVERTENCIAS:** 

* Al ejecutar la siguiente celda el servidor se inciará desde la notebook, por lo que para ejecutar cualquier otra celda es necesario interrumpir la ejecución del kernel.

* Asegúrese que no haya otro servicio escuchando en el puerto *5000*.

In [ ]:
!./manage.py runserver 0.0.0.0:5000

http://localhost:5000/api/1221

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2019.</p>